In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = context.catalog.load("foehn_data_cleansed")

In [ ]:
for col in df.filter(regex="foehn"):
    print(df[col].value_counts(dropna=False))

In [ ]:
df.max(axis=0)[:-1].sort_values(ascending=False)[0:50]

In [ ]:
print(len(df.index))

In [ ]:
features = set(df.columns.tolist()) - set(["date"])
df.dropna(how="all", axis=0, subset=features)